In [6]:
from google.cloud import bigquery
import os
from time import time

#!export GOOGLE_APPLICATION_CREDENTIALS="/Users/th/Documents/Kaggle/jupyternotebook-353502-90e27b6f759e.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/th/Documents/Kaggle/jupyternotebook-353502-90e27b6f759e.json"

# JOINs and UNIONs

vamos trabalhar com o dataset "hacker_news" aqui vamos revisar as 5 linhas da tabela 'comments'

In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project= "bigquery-public-data")

# API  request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,<NA>,<NA>,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,<NA>,<NA>,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,<NA>,<NA>,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,<NA>,<NA>,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,<NA>,<NA>,0


vamos trabalhar também com a tabela 'stories'

In [5]:
# Construct a reference to the "stories" table
table_ref = dataset_ref.table("stories")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,by,score,time,time_ts,title,url,text,deleted,dead,descendants,author
0,6940813,sarath237,0,1387536270,2013-12-20 10:44:30+00:00,Sheryl Brindo Hot Pics,http://www.youtube.com/watch?v=ym1cyxneB0Y,Sheryl Brindo Hot Pics,<NA>,True,<NA>,sarath237
1,6991401,123123321321,0,1388508751,2013-12-31 16:52:31+00:00,Are you people also put off by the culture of ...,,They&#x27;re pretty explicitly &#x27;startup f...,<NA>,True,<NA>,123123321321
2,1531556,ssn,0,1279617234,2010-07-20 09:13:54+00:00,New UI for Google Image Search,http://googlesystem.blogspot.com/2010/07/googl...,Again following on Bing's lead.,<NA>,<NA>,0,ssn
3,5012398,hoju,0,1357387877,2013-01-05 12:11:17+00:00,Historic website screenshots,http://webscraping.com/blog/Generate-website-s...,Python script to generate historic screenshots...,<NA>,<NA>,0,hoju
4,7214182,kogir,0,1401561740,2014-05-31 18:42:20+00:00,Placeholder,,Mind the gap.,<NA>,<NA>,0,kogir


A seguir vemos um exemplo de query que puxa informações tanto da tabela 'comments' e 'stories' para criar uma tabela mostrando todos os stories postados em 01/01/2012

In [11]:
# Query para selecionar todos os stories postados em primeiro de janeiro de 2012 com o número de comentários
join_query = """
             WITH c AS
             (
             SELECT parent, COUNT(*) as num_comments
             FROM `bigquery-public-data.hacker_news.comments`
             GROUP BY parent
             )
             SELECT s.id as story_id, s.by, s.title, c.num_comments
             FROM `bigquery-public-data.hacker_news.stories` AS s
             LEFT JOIN c
             ON s.id = c.parent
             WHERE EXTRACT(DATE FROM s.time_ts) = '2012-01-01'
             ORDER BY c.num_comments DESC
             """

# Rodar a query e retornar um dataframe pandas
join_result = client.query(join_query).result().to_dataframe()
join_result.head()

,story_id,by,title,num_comments
0,3412900,whoishiring,Ask HN: Who is Hiring? (January 2012),154
1,3412901,whoishiring,Ask HN: Freelancer? Seeking freelancer? (Janua...,97
2,3412643,jemeshsu,Avoid Apress,30
3,3414012,ramanujam,Impress.js - a Prezi like implementation using...,27
4,3412891,Brajeshwar,"There's no shame in code that is simply ""good ...",27


In [12]:
# Stores que receberam nenhum comentáRecursionError
join_result.tail()

,story_id,by,title,num_comments
439,3413408,TheYoungEagle,Free Download The Sims 3 Pc Games,<NA>
440,3412632,UsedCarFleetCom,Used Car fleet,<NA>
441,3412940,julelara,Washington Redskins vs Philadelphia Eagles liv...,<NA>
442,3412327,arroyo,Can your business card be followed?,<NA>
443,3413203,norris_tony44,10 Popular iPhone Games you Must Play,<NA>


agora vamos escrever uma query que retorna todos os 'usernames' que postaram 'stories' ou 'comments'em primeiro de janeiro de 2014. Nesta usaremos UNION DISTINCT ao invés de

UNION ALL para assegurar que cada uruário apareça pelo menos uma vez na consulta

In [18]:
# Query para selecionar todos os usuários que postaram stories ou comments em primeiro de janeiro de 2014
union_query =   """
              SELECT c.by
              FROM `bigquery-public-data.hacker_news.comments` AS c
              WHERE EXTRACT(DATE FROM c.time_ts) = '2014-01-01'
              UNION DISTINCT
              SELECT s.by
              FROM `bigquery-public-data.hacker_news.stories` AS s
              WHERE EXTRACT(DATE FROM s.time_ts) = '2014-01-01'
              """

# Rodar a query e retornar um dataframe pandas
union_result = client.query(union_query).result().to_dataframe()
union_result.head()

,by
0,yapatel
1,shawndumas
2,SPYBOT
3,rpm4321
4,jeanbebe


Para conseguir o número de usuários que postaram em primeiro de janeiro de 2014 precisamos pegar o len do dataframe

In [19]:
len(union_result)

2282

## Exercício

In [21]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "posts_questions" table
table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,320268,Html.ActionLink doesn’t render # properly,<p>When using Html.ActionLink passing a string...,<NA>,0,0,NaT,2008-11-26 10:42:37.477000+00:00,0,2009-02-06 20:13:54.370000+00:00,NaT,None,<NA>,Paulo,<NA>,None,1,0,asp.net-mvc,380
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,<NA>,0,0,NaT,2008-11-27 15:12:37.497000+00:00,0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288,None,41000,None,1,0,haskell|lambda|functional-programming|lambda-c...,488
2,390605,While vs. Do While,<p>I've seen both the blocks of code in use se...,390608,0,0,NaT,2008-12-24 01:49:54.230000+00:00,2,2008-12-24 03:08:55.897000+00:00,NaT,None,<NA>,Unkwntech,115,None,1,0,language-agnostic|loops,11086
3,413246,Protect ASP.NET Source code,<p>Im currently doing some research in how to ...,<NA>,0,0,NaT,2009-01-05 14:23:51.040000+00:00,0,2009-03-24 21:30:22.370000+00:00,2009-01-05 14:42:28.257000+00:00,Tom Anderson,13502,Velnias,<NA>,None,1,0,asp.net|deployment|obfuscation,4811
4,454921,"Difference between ""int[] myArray"" and ""int my...",<blockquote>\n <p><strong>Possible Duplicate:...,454928,0,0,NaT,2009-01-18 10:22:52.177000+00:00,0,2009-01-18 10:30:50.930000+00:00,2017-05-23 11:49:26.567000+00:00,None,-1,Evan Fosmark,49701,None,1,0,java|arrays,795


take a look at the 'posts_answers' table

In [22]:
# Construct a reference to the "posts_answers" table
table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,18,None,<p>For a table like this:</p>\n\n<pre><code>CR...,None,None,2,NaT,2008-08-01 05:12:44.193000+00:00,None,2016-06-02 05:56:26.060000+00:00,2016-06-02 05:56:26.060000+00:00,Jeff Atwood,126039,phpguy,<NA>,17,2,59,None,None
1,165,None,"<p>You can use a <a href=""http://sharpdevelop....",None,None,0,NaT,2008-08-01 18:04:25.023000+00:00,None,2019-04-06 14:03:51.080000+00:00,2019-04-06 14:03:51.080000+00:00,None,1721793,user2189331,<NA>,145,2,10,None,None
2,1028,None,<p>The VB code looks something like this:</p>\...,None,None,0,NaT,2008-08-04 04:58:40.300000+00:00,None,2013-02-07 13:22:14.680000+00:00,2013-02-07 13:22:14.680000+00:00,None,395659,user2189331,<NA>,947,2,8,None,None
3,1073,None,<p>My first choice would be a dedicated heap t...,None,None,0,NaT,2008-08-04 07:51:02.997000+00:00,None,2015-09-01 17:32:32.120000+00:00,2015-09-01 17:32:32.120000+00:00,None,45459,user2189331,<NA>,1069,2,29,None,None
4,1260,None,<p>I found the answer. all you have to do is a...,None,None,0,NaT,2008-08-04 14:06:02.863000+00:00,None,2016-12-20 08:38:48.867000+00:00,2016-12-20 08:38:48.867000+00:00,None,1221571,Jin,<NA>,1229,2,1,None,None


In [23]:
# Query que verifica em quantos segundos demora para uma pergunta ser respondida

first_query = """
            SELECT q.id AS q_id,
                MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) AS time_to_answer
            FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers`AS a
            ON q.id = a.parent_id
            WHERE q.creation_date >= '2018-01-01' AND q.creation_date < '2018-02-01'
            GROUP BY q_id
            ORDER BY time_to_answer
            """

first_result = client.query(first_query).result().to_dataframe()
print("Percentage of answerd questions: %s%%" % \
        (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Number of questions:", len(first_result))
first_result.head()

Percentage of answerd questions: 100.0%
Number of questions: 134731


,q_id,time_to_answer
0,48382183,-132444692
1,48350506,0
2,48292208,0
3,48091268,0
4,48101715,0


# 1)

In [24]:
# Corrigindo a query para o JOIN correto para retornar o número correto de perguntas respondidas

first_query = """
            SELECT q.id AS q_id,
                MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) AS time_to_answer
            FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers`AS a
            ON q.id = a.parent_id
            WHERE q.creation_date >= '2018-01-01' AND q.creation_date < '2018-02-01'
            GROUP BY q_id
            ORDER BY time_to_answer
            """

first_result = client.query(first_query).result().to_dataframe()
print("Percentage of answerd questions: %s%%" % \
        (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Number of questions:", len(first_result))
first_result.head()

Percentage of answerd questions: 83.25104889487571%
Number of questions: 161837


,q_id,time_to_answer
0,48545029,<NA>
1,48058963,<NA>
2,48046972,<NA>
3,48320296,<NA>
4,48289335,<NA>


# 2) 
Query para consultar usuários que fizeram perguntas e também responderam perguntas

In [27]:
q_and_a_query = """
                SELECT q.owner_user_id AS owner_user_id,
                    MIN(q.creation_date) AS q_creation_date,
                    MIN(a.creation_date) AS a_creation_date
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                ON q.owner_user_id = a.owner_user_id 
                WHERE q.creation_date >= '2019-01-01' AND q.creation_date < '2019-02-01' 
                    AND a.creation_date >= '2019-01-01' AND a.creation_date < '2019-02-01'
                GROUP BY owner_user_id
                """
q_and_a_result = client.query(q_and_a_query).result().to_dataframe()
q_and_a_result.head()

,owner_user_id,q_creation_date,a_creation_date
0,10515677,2019-01-01 22:20:16.900000+00:00,2019-01-01 03:38:21.560000+00:00
1,7294900,2019-01-01 04:14:42.093000+00:00,2019-01-01 09:23:44.083000+00:00
2,7092472,2019-01-26 05:43:44.177000+00:00,2019-01-01 09:28:07.277000+00:00
3,7208364,2019-01-09 04:38:41.117000+00:00,2019-01-01 12:41:06.293000+00:00
4,6943737,2019-01-01 05:36:39.403000+00:00,2019-01-01 12:43:07.357000+00:00


# 3)

In [44]:
three_tables_query = """
                    SELECT u.id AS id,
                        MIN(q.creation_date) AS q_creation_date,
                        MIN(a.creation_date) AS a_creation_date
                    FROM `bigquery-public-data.stackoverflow.users` AS u
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
                        ON u.id = q.owner_user_id
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                        ON u.id = a.owner_user_id
                    WHERE u.creation_date >= '2019-01-01' AND u.creation_date < '2019-02-01'
                    GROUP BY id
                    """

three_tables_result = client.query(three_tables_query).result().to_dataframe()
three_tables_result.head()

,id,q_creation_date,a_creation_date
0,10984606,NaT,NaT
1,10907660,2019-01-13 11:49:54.267000+00:00,NaT
2,10922269,NaT,NaT
3,10907075,NaT,NaT
4,10932334,2019-01-18 08:36:30.617000+00:00,NaT


# 4)
quantos usuários distintos postaram em primeiro de janeiro 2019

In [57]:
all_users_query = """
                    SELECT q.owner_user_id AS owner_user_id,
                    FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
                    UNION DISTINCT 
                    SELECT a.owner_user_id
                    FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
                    WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
                    """

all_users_result = client.query(all_users_query).result().to_dataframe()
all_users_result.head()

,owner_user_id
0,7918089
1,3049065
2,10770784
3,353130
4,4115031


# Analytic Functions

In [11]:
# Craete a "Client" object
client = bigquery.Client()

# Construct a reference to the "asn_francisco"dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,944732,2618,2015-09-24 17:22:00+00:00,Mezes,83,2015-09-24 18:06:00+00:00,Mezes,83,653,94063,Customer
1,984595,5957,2015-10-25 18:12:00+00:00,Mezes,83,2015-10-25 19:51:00+00:00,Mezes,83,52,nil,Customer
2,984596,5913,2015-10-25 18:13:00+00:00,Mezes,83,2015-10-25 19:51:00+00:00,Mezes,83,121,nil,Customer
3,1129385,6079,2016-03-18 10:33:00+00:00,Mezes,83,2016-03-18 12:14:00+00:00,Mezes,83,208,94070,Customer
4,1030383,5780,2015-12-06 10:52:00+00:00,Mezes,83,2015-12-06 12:28:00+00:00,Mezes,83,44,94064,Customer


cada linha da tabela corresponde a uma viagem de bicicleta diferente, e nos podemos usar as funções analíticas para calcular o valor acumulado das viagens para cada data de 2015

In [9]:
# Query to count the (cumulative) number of trips per day
num_trips_query = """
                WITH trips_by_day AS
                (
                SELECT DATE(start_date) AS trip_date,
                    COUNT(*) AS num_trips
                FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                WHERE EXTRACT(YEAR FROM start_date) = 2015
                GROUP BY trip_date
                )
                SELECT *,
                    SUM(num_trips)
                        OVER(
                            ORDER BY trip_date
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                            ) AS cumulative_trips
                    FROM trips_by_day
                """

# Run the query and return a pandas dataframe
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284


In [14]:
# Query to track begining and ending stations on October 25, 2015, for each bike

start_end_query = """
                SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25'
                """

# Run the query, and return a pandas Dataframe
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,294,14:29:00,64,77,64,77
1,349,14:37:00,42,51,42,75
2,349,18:01:00,42,51,75,51
3,402,17:18:00,77,75,77,65
4,402,17:42:00,77,75,65,75


# Exercício

In [3]:
client = bigquery.Client()

dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("taxi_trips")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,7b3e74418111fef19fd73835853d9c1cebd727d5,fa62820eb95bd0b9cc1a0aaf6cf26730aee22ec578db91...,2014-04-01 16:00:00+00:00,2014-04-01 16:00:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,6.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,9d2eefe122d32a385207087a71dd33deed0eea96,89612cda57630834fb854b4e42206d3478145557d975ce...,2014-03-06 21:30:00+00:00,2014-03-06 21:30:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,12.05,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
2,a8410244d022d258ae357f6ede25d0162efd675f,36dee7e88e2048dfeb8a98c9e420a1600544a8d4f21fe6...,2014-04-18 10:45:00+00:00,2014-04-18 10:45:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,12.55,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,c590e1ea0a0998541e049de913e4a26499955123,85d86e614d844575634820c6e19498d1cfd59bdecc396b...,2015-02-05 15:30:00+00:00,2015-02-05 15:45:00+00:00,1020,2.5,<NA>,<NA>,<NA>,<NA>,...,0.0,9.85,Cash,None,NaN,NaN,None,NaN,NaN,None
4,bdd767ee32e7038e4c1bcac28cd516cc7cb549b8,f4bc2acfbb68d505b7ca37f8553ab1fc28a2686429b55d...,2014-03-28 09:45:00+00:00,2014-03-28 09:45:00+00:00,0,0.0,<NA>,<NA>,<NA>,<NA>,...,0.0,16.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None


# 1)

In [18]:
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                        """

avg_num_trips_result = client.query(avg_num_trips_query).result().to_dataframe()
avg_num_trips_result.head()

,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


# 2)

In [7]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER(
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                                ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()
trip_number_result.head()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,38,2017-05-01 01:45:00+00:00,2017-05-01 02:00:00+00:00,1
1,38,2017-05-01 03:30:00+00:00,2017-05-01 03:45:00+00:00,2
2,38,2017-05-01 06:30:00+00:00,2017-05-01 06:45:00+00:00,3
3,38,2017-05-01 07:45:00+00:00,2017-05-01 08:15:00+00:00,4
4,38,2017-05-01 08:00:00+00:00,2017-05-01 08:00:00+00:00,5


# 3)

In [9]:
break_time_query = """
                    SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) AS prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

break_time_result = client.query(break_time_query).result().to_dataframe()
break_time_result.head()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,056d6f1b203358cf84922075a99e40eb045f187b65ce59...,2017-05-01 04:45:00+00:00,2017-05-01 05:15:00+00:00,<NA>
1,12892fdf027712c46f330164331346d04cd435d0ff7020...,2017-05-01 00:15:00+00:00,2017-05-01 00:30:00+00:00,<NA>
2,12892fdf027712c46f330164331346d04cd435d0ff7020...,2017-05-01 23:45:00+00:00,2017-05-02 00:00:00+00:00,60
3,1be8760476f6abd128dfc50870dce6e97788982a61748f...,2017-05-01 04:45:00+00:00,2017-05-01 05:00:00+00:00,<NA>
4,1be8760476f6abd128dfc50870dce6e97788982a61748f...,2017-05-01 05:15:00+00:00,2017-05-01 05:15:00+00:00,15


# Nested and Repeated Data

In [11]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "google_ana;utics_sample" dataset
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("ga_sessions_20170801")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501583974,1501583974,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Car...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2248281639583218707,None,None,Organic Search,Not Socially Engaged
1,<NA>,1,1501616585,1501616585,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8647436381089107732,None,None,Organic Search,Not Socially Engaged
2,<NA>,1,1501583344,1501583344,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2055839700856389632,None,None,Organic Search,Not Socially Engaged
3,<NA>,1,1501573386,1501573386,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 0, 'minut...",0750846065342433129,None,None,Direct,Not Socially Engaged
4,<NA>,8,1501651467,1501651467,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",0573427169410921198,None,None,Organic Search,Not Socially Engaged


In [12]:
# printando o schema

print("SCHEMA field for the 'totals' column:\n")
print(table.schema[5])

print("\nSCHEMA field for the 'device' column:\n")
print(table.schema[7])

SCHEMA field for the 'totals' column:

SchemaField('totals', 'RECORD', 'NULLABLE', None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('hits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('timeOnScreen', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('totalTransactionRevenue', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('sessionQualityDim', 'INTEGER', 'NULLABLE', None, (), None)), None)

SCHEMA field for th

In [14]:
# Query para contar o número de transactions por browser
query = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) AS total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """

# rodando a query e visualizando os 5 primeiros resultados
result = client.query(query).result().to_dataframe()
result.head()

,device_browser,total_transactions
0,Chrome,41
1,Safari,3
2,Firefox,1
3,Internet Explorer,<NA>
4,UC Browser,<NA>


In [15]:
# Query para determinar o landing point mais popular

query = """
        SELECT hits.page.pagePath AS path,
            COUNT(hits.page.pagePath) AS counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`,
            UNNEST(hits) AS hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

result = client.query(query).result().to_dataframe()
result.head()

,path,counts
0,/home,1257
1,/google+redesign/shop+by+brand/youtube,587
2,/google+redesign/apparel/mens/mens+t+shirts,117
3,/signin.html,78
4,/basket.html,35


# Exercício

In [3]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "sample_commits" table
table_ref = dataset_ref.table("sample_commits")

# API request - fetch the table
sample_commits_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(sample_commits_table, max_results=5).to_dataframe()

,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,3eca86e75ec7a7d4b9a9c8091b11676f7bd2a39f,8e1b4380409a85a922ee0d3f622b5dd4d16bcfad,[104a0c02e8b1936c049e18a6d4e4ab040fb61213],"{'name': 'Mark Rutland', 'email': '1db9dd262be...","{'name': 'Catalin Marinas', 'email': '15ce75b2...",arm64: Remove fixmap include fragility,arm64: Remove fixmap include fragility\n\nThe ...,"[{'key': 'Signed-off-by', 'value': 'Mark Rutla...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
1,7158627686f02319c50c8d9d78f75d4c8d126ff2,3b4d781bd966f07cad1b67b137f0ff8b89430e9a,[66aa8d6a145b6a66566b4fce219cc56c3d0e01c3],"{'name': 'Will Deacon', 'email': 'b913f13ef92a...","{'name': 'Catalin Marinas', 'email': '15ce75b2...",arm64: percpu: implement optimised pcpu access...,arm64: percpu: implement optimised pcpu access...,"[{'key': 'Signed-off-by', 'value': 'Will Deaco...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
2,9732cafd9dc0206479be919baf0067239f0a63ca,c8878035ac9cb6dce592957f12dc1723a583989d,[f3c003f72dfb2497056bcbb864885837a1968ed5],"{'name': 'Jiang Liu', 'email': 'c745fa7b96fe79...","{'name': 'Catalin Marinas', 'email': '15ce75b2...","arm64, jump label: optimize jump label impleme...","arm64, jump label: optimize jump label impleme...","[{'key': 'Reviewed-by', 'value': 'Will Deacon ...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
3,4702abd3f9728893ad5b0f4389e1902588510459,32926e7c55ef585d9b9c174a0e5f9ed13ed6bf7e,[ddf28352b80c86754a6424e3a61e8bdf9213b3c7],"{'name': 'Nicolas Pitre', 'email': '408789a210...","{'name': 'Arnd Bergmann', 'email': 'f2c659f019...",ARM: mach-nuc93x: delete,ARM: mach-nuc93x: delete\n\nThis architecture ...,"[{'key': 'Signed-off-by', 'value': 'Nicolas Pi...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
4,57bd4b91a6cfc5bad4c5d829ef85293ea63643ea,2ffc2066eb7638e185663e9d849663403229d4e5,[f74c95c20bad8e183e41283475f68a3e7b247af4],"{'name': 'Ben Dooks', 'email': '1177f64998f284...","{'name': 'Ben Dooks', 'email': '1177f64998f284...",[ARM] S3C24XX: Movev udc headers to arch/arm/p...,[ARM] S3C24XX: Movev udc headers to arch/arm/p...,"[{'key': 'Signed-off-by', 'value': 'Ben Dooks ...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None


In [17]:
# Printando o schema
sample_commits_table.schema

[SchemaField('commit', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('tree', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('parent', 'STRING', 'REPEATED', None, (), None),
 SchemaField('author', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('committer', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('subject', 'STRING', 'NULLABLE', None, (), None),
 SchemaFie

# 1) Query para retornar o usuário com mais commit no ano de 2016 do github

In [19]:
max_commits_query = """
                    SELECT committer.name AS committer_name,
                        COUNT(*) AS num_commits
                    FROM `bigquery-public-data.github_repos.sample_commits`
                    WHERE EXTRACT(YEAR FROM committer.date) = 2016
                    GROUP BY committer_name
                    ORDER BY num_commits DESC
                    """

max_commits_result = client.query(max_commits_query).result().to_dataframe()
max_commits_result.head()

,committer_name,num_commits
0,Greg Kroah-Hartman,3545
1,David S. Miller,3120
2,TensorFlower Gardener,2449
3,Linus Torvalds,2424
4,Benjamin Pasero,1127


# 2) Olhando para tabela 'languages'

In [21]:
table_ref = dataset_ref.table("languages")

languages_table = client.get_table(table_ref)

client.list_rows(languages_table, max_results=5).to_dataframe()

,repo_name,language
0,flaviomahooney/prog1_2016,"[{'name': 'C', 'bytes': 15056}]"
1,raidenawkward/ipc_samples,"[{'name': 'C', 'bytes': 11448}]"
2,zhang3/asm018x,"[{'name': 'C', 'bytes': 246}]"
3,geekofhearts/lang,"[{'name': 'C', 'bytes': 2034}]"
4,84rn/pokerbot,"[{'name': 'C', 'bytes': 41424}]"


In [22]:
# Printando o schema da tabela languages
languages_table.schema

[SchemaField('repo_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('language', 'RECORD', 'REPEATED', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('bytes', 'INTEGER', 'NULLABLE', None, (), None)), None)]

# 3) Query para retornar qual linguagem de programação é a mais popular no github

In [24]:
pop_lang_query = """
                SELECT l.name AS language_name,
                    COUNT(*) AS num_repos
                FROM `bigquery-public-data.github_repos.languages`,
                    UNNEST(language) AS l
                GROUP BY language_name
                ORDER BY num_repos DESC
                """

pop_lang_result = client.query(pop_lang_query).result().to_dataframe()
pop_lang_result.head()

,language_name,num_repos
0,JavaScript,1099822
1,CSS,808800
2,HTML,777598
3,Shell,639826
4,Python,550020


# 4) qual linguagem é utilizada no respositório 'polyrabbit/polyglot' com mais linguages?

In [27]:
all_lang_query = """
                SELECT l.name AS name,
                    l.bytes AS bytes
                FROM `bigquery-public-data.github_repos.languages`,
                    UNNEST(language) AS l
                WHERE repo_name = 'polyrabbit/polyglot'
                ORDER BY bytes DESC
                """

all_lang_result = client.query(all_lang_query).result().to_dataframe()
all_lang_result.head()

,name,bytes
0,Lasso,834726
1,C,819142
2,Mercury,709952
3,Objective-C,495392
4,Game Maker Language,298131


# Writing Efficient Queries

In [4]:
client = bigquery.Client()

def show_amount_of_data_scanned(query):
    # dry_run lets us see how much data the query uses without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job = client.query(query, job_config=dry_run_config)
    print('Data processed: {} GB'.format(round(query_job.total_bytes_processed / 10**9, 3)))

def show_time_to_run(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print('Time to run: {} seconds'.format(round(end-start, 3)))


Sempre selecione apenas as colunas que se deseja

In [31]:
star_query = "SELECT * FROM `bigquery-public-data.github_repos.contents`"
show_amount_of_data_scanned(star_query)

basic_query = "SELECT size, binary FROM `bigquery-public-data.github_repos.contents`"
show_amount_of_data_scanned(basic_query)

Data processed: 2650.24 GB
Data processed: 2.51 GB


lendo menos dados

In [35]:
more_data_query = """
                    SELECT MIN(start_station_name) AS start_station_name,
                      MIN(end_station_name) AS end_station_name,
                      AVG(duration_sec) AS avg_duration_sec
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_id != end_station_id 
                  GROUP BY start_station_id, end_station_id
                  LIMIT 10
                    """

show_amount_of_data_scanned(more_data_query)

less_data_query = """
                    SELECT start_station_name,
                      end_station_name,
                      AVG(duration_sec) AS avg_duration_sec                  
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_name != end_station_name
                  GROUP BY start_station_name, end_station_name
                  LIMIT 10
                    """

show_amount_of_data_scanned(less_data_query)

Data processed: 0.076 GB
Data processed: 0.06 GB


Evitando JOINs N:N

In [7]:
big_join_query = """
                SELECT repo,
                     COUNT(DISTINCT c.committer.name) as num_committers,
                     COUNT(DISTINCT f.id) AS num_files
                 FROM `bigquery-public-data.github_repos.commits` AS c,
                     UNNEST(c.repo_name) AS repo
                 INNER JOIN `bigquery-public-data.github_repos.files` AS f
                     ON f.repo_name = repo
                 WHERE f.repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                 GROUP BY repo
                 ORDER BY repo
                """

show_time_to_run(big_join_query)

small_join_query = """
                   WITH commits AS
                   (
                   SELECT COUNT(DISTINCT committer.name) AS num_committers, repo
                   FROM `bigquery-public-data.github_repos.commits`,
                       UNNEST(repo_name) as repo
                   WHERE repo IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   ),
                   files AS 
                   (
                   SELECT COUNT(DISTINCT id) AS num_files, repo_name as repo
                   FROM `bigquery-public-data.github_repos.files`
                   WHERE repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   )
                   SELECT commits.repo, commits.num_committers, files.num_files
                   FROM commits 
                   INNER JOIN files
                       ON commits.repo = files.repo
                   ORDER BY repo
                   """

show_time_to_run(small_join_query)


Time to run: 25.746 seconds
Time to run: 12.768 seconds
